# Проект по SQL
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.\
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цели исследования**

1. Определить количество книг, выпущенных после 1 января 2000 года, чтобы понять объем литературного контента, доступного в сервисе.
2. Рассчитать количество обзоров и среднюю оценку для каждой книги, чтобы оценить их популярность среди пользователей.
3. Идентифицировать издательство, которое выпустило наибольшее количество книг, толщиной более 50 страниц, чтобы определить влиятельных партнеров и исключить неподходящие форматы (например, брошюры).
4. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, чтобы выявить наиболее успешных авторов среди пользователей.
5. Рассчитать среднее количество обзоров от пользователей, которые поставили более 48 оценок, чтобы оценить активность и вовлеченность этой группы пользователей.

Результаты анализа помогут сформулировать ценностное предложение для нового продукта, а также определить наиболее популярные книги, авторов и издательства, которые могут быть привлекательными для пользователей сервиса.

## Импорт библиотек, подключение к СУБД и знакомство с данными

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': '11111', # имя пользователя
'pwd': '22222', # пароль
'host': 'blablabla.com',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Схема данных**

![Схема данных](https://pictures.s3.yandex.net/resources/scheme_1589269096.png)

**Описание данных**

**Таблица `books`**\
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**\
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**\
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**\
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**\
Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

## Анализ БД
### Количество книг, выпущенных после 1 января 2000 года

In [7]:
query = '''SELECT COUNT(*) AS book_count
FROM books
WHERE publication_date > '2000-01-01';
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_count
0,819


После 1 января 2000 года было выпущено 819 книг.

### Количество обзоров и средняя оценка для каждой книги

In [11]:
query = '''SELECT
  b.book_id,
  b.title,
  (
    SELECT COUNT(*)
    FROM reviews r
    WHERE r.book_id = b.book_id
  ) AS review_count,
  (
    SELECT AVG(ra.rating)
    FROM ratings ra
    WHERE ra.book_id = b.book_id
  ) AS average_rating
FROM books b
ORDER BY review_count DESC;
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,title,review_count,average_rating
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Самое большое количество обзоров сделали для книги `Twilight (Twilight #1)`.

### Издательство, которое выпустило наибольшее количество книг, толщиной более 50 страниц

In [13]:
query = '''SELECT p.publisher_id, p.publisher, COUNT(*) AS book_count
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY book_count DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,book_count
0,212,Penguin Books,42


Наибольшее количество книг, толщиной более 50 страниц было выпущено издательством `Penguin Books`.

### Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками)

In [14]:
query = '''SELECT a.author_id, a.author, AVG(rating) AS average_rating
FROM books b
JOIN authors a ON b.author_id = a.author_id
JOIN (
    SELECT book_id, AVG(rating) AS rating
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(*) >= 50
) AS subquery ON b.book_id = subquery.book_id
GROUP BY a.author_id, a.author
ORDER BY average_rating DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка книг у автора `J.K. Rowling/Mary GrandPré`.

### Среднее количество обзоров от пользователей, которые поставили более 48 оценок

In [15]:
query = '''SELECT AVG(review_count) AS average_review_count
FROM (
    SELECT username, COUNT(review_id) AS review_count
    FROM reviews
    WHERE username IN (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating_id) > 48
    )
    GROUP BY username
) AS review_counts;
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,average_review_count
0,24.0


Пользователи, которые поставили более 48 оценок в среднем сделали 24 обзора.

## Результаты анализа БД

1. После 1 января 2000 года было выпущено `819 книг`.
2. Самое большое количество обзоров сделали для книги `Twilight (Twilight #1)`.
3. Наибольшее количество книг, длиной более 50 страниц было выпущено издательством `Penguin Books`.
4. Самая высокая средняя оценка книг у автора `J.K. Rowling/Mary GrandPré`.
5. Пользователи, которые поставили более 48 оценок в среднем сделали `24 обзора`.